# Exercicio 2
## Criar uma cifra com autenticação de meta-dados a partir de um PRG


# Alínea a.

Como objetivo inicial, esperava-se que o nosso grupo implementasse um gerador pseudo-aleatório do usando uma função XOF("extended output function") - SHAKE256 - de forma a gerar uma sequência de palavras, cada uma com 64 bits.


Para começar, criamos uma função que recebe a password e a partir de uma *KDF* - no nosso caso decidimos escolher a **PBKDF2HMAC**, mas também poderíamos ter usado a *HKD*. Esta decisão foi tida em conta com a leitura da documentação da lib *Crypthography*, a qual inferia que há diferentes tipos de funções de derivação para diferentes tipos de propósitos. Como o nosso criar uma seed para o gerador, escolhemos um dos que era recomendado. 

In [ ]:
def derive_key(password):
    salt = os.urandom(16)
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        )

    return kdf.derive(password)

De seguida, e conforme enunciado, era necessário criar o gerador, que tivesse um limite de palavras definido por um valor inicialmente parametrizado como **N**. Com a seed gerada pela *key derivation function* que escolhemos seria possível gerar 2^N palavras de 64 bits cada.
Seguindo o requisitado para este projeto, era necessário usar o **SHAKE256** para definir o gerador - como se sabe o *SHAKE256* é uma *extendable-output function* e define-se o seu uso tendo em conta  . 


In [ ]:
def prg(seed,N):
    digest = hashes.Hash(hashes.SHAKE256(8 * pow(2,N))) # sequencia palavras 64 bits / 8 = 8 bytes
    digest.update(seed)
    msg = digest.finalize()
    return msg

A partir da sequência de bits representativa da lista de palavras, gerada pelo PRG, conseguimos aplicar a cifra **OTP**. Assumindo a variante construída em 1919, é necessário definir a operação *XOR* em contraste com a sua versão original, criada com adição modular. É então preciso aplicar esta operação    


In [ ]:
def cipher(k,msg):
    ciphertext = b''
    pad = padding.PKCS7(64).padder()
    
    # adds padding to the last block of bytes of the message -> this garantees that the block size is multiple
    # basically stuffs the last block with pad chars 
    padded = pad.update(msg) + pad.finalize()
    # mesage is divided in blocks of 8 bytes
    p = pad_divide(padded)

    for x in range (len(p)): # Percorre blocos do texto limpo
        for bloco, byte in enumerate(p[x]): # Percorre bytes do bloco do texto limpo
            ciphertext += bytes([byte ^ k[x:(x+1)*BLOCK][bloco]]) # xor of 2 bit sequences plain text and cipher_key
    return ciphertext


In [ ]:

def decipher(k,ciphertext):
    plaintext=b''
    
    p=pad_divide(ciphertext)

    for x in range (len(p)): # Percorre blocos do texto cifrado
        for bloco, byte in enumerate(p[x]): # Percorre bytes do bloco do texto cifrado
            plaintext += bytes([byte ^ k[x:(x+1)*BLOCK][bloco]]) 
    
    # Algoritmo para retirar padding para decifragem
    unpadder = padding.PKCS7(64).unpadder()
    # Retira bytes adicionados 
    unpadded = unpadder.update(plaintext) + unpadder.finalize()
    return unpadded.decode("utf-8")